In [ ]:
from detector import *
from flask import Flask, render_template, Response

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')

In [ ]:
@app.route('/video_feed')
def gen_frames():
    #Video streaming route. Put this in the src attribute of an img tag
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

In [ ]:
if __name__ == '__main__':
    app.run(port=2204)